In [609]:
### Extra code to reuse

## Exporting data to CSV
# combined_data.to_csv('file_name.csv')

## Finding what the Data Type a column is.
# data_type = IMDb_Basics['isAdult'].dtype
# print(data_type)

## Filtering dataframes for a specific values
# filtered_df = IMDB_Basics[IMDB_Basics['primaryTitle'].str.contains('Bond', case=False, na=False)]
# print(filtered_df)

# Optionally save the combined dataframe to a new CSV file
# output_file = path_to_files + 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)
# print(f"Combined CSV saved to {output_file}")

# Code to troubleshoot date columns

# 1. Check for NaT values:
# If you used errors='coerce' during the conversion, you can check for NaT values, which would indicate conversion failures.
# nat_counts = all_stocks['Date'].isna().sum()
# print(f"Number of NaT values in 'Date': {nat_counts}")

# 2. Check the range of dates:
# Look at the minimum and maximum values to ensure they are within expected bounds.
# print("Earliest date:", all_stocks['Date'].min())
# print("Latest date:", all_stocks['Date'].max())

# 3. Sample random dates:
# Random sampling of dates can give you a quick spot check to see if dates look correctly formatted.
# print(all_stocks['Date'].sample(10))

# 4. Validate with known dates:
# If you know certain dates should exist in the dataset, check those specifically.
# known_dates = ["1997-05-15", "2005-07-08"]  # Example dates you expect to find
# for date in known_dates:
#     if pd.to_datetime(date) not in all_stocks['Date'].values:
#         print(f"Date {date} is not found or incorrectly formatted.")
#     else:
#         print(f"Date {date} is correctly formatted.")

# 5. Check for duplicates:
# Ensuring that there are no duplicate dates which could suggest an issue with the conversion.
# duplicate_dates = all_stocks['Date'].duplicated().sum()
# print(f"Number of duplicate dates: {duplicate_dates}")

## Remove duplicate rows based on the 'Release' column, keeping the first occurrence
# no_duplicates = no_duplicates.drop_duplicates(subset='Release', keep='first')

# # Remove the dollar sign and commas, then convert the column to numeric values
# no_duplicates['Gross'] = no_duplicates['Gross'].replace(r'[\$,]', '', regex=True).astype(float)
# # Calculate the mean of the 'Gross' column after conversion
# gross_mean = no_duplicates['Gross'].mean()
# # Now 'gross_mean' contains the average of the 'Gross' column as a float
# print(gross_mean)

<span style="color:green">

# Meeting notes

## Meeting 1 Feedback- In-Progress
Project Proposal Meeting notes
 Outcome variable is Stock returns
We cannot have 2 different observations we could use weighted average or just an average of the 2 movies. Because in a month there could be 2 movies launched by Disney and we need to be able to see what is the causal but with just 1 movie.
We should do quarterly and then do monthly. So this way we will be able to see a change in the response from investors and the market.
I will determine the number of years of data according to the number of observations.
(I can include Text analysis based on individual people’s reviews for each movies. I can use this for my Capstone project. I can do analysis on if someone left a good review score and bad text comments.)

## Meeting 2 Feedback- 
- I will do my analysis on month to month basis on the stock prices of companies.

- Y is the stock price a month after the release minus the stock price at the month of the release.
- As a control I can include, Vanguard 500 and also the Media Company Index. I can include the changes in it than the absolute values.
- 

Tasks done
- For the economic indicators the data spans for each quarter in a year and not monthly. What do we do about that?
    - We can duplicate the same value for the next 3 months.
- For the Stocks dataset what columns do we need? Do we need open, close? This way I can reduce the size of the model.
    - Use the close price for all companies.
- In the IMDb final table I still need to add the release date.
    - Get the format to be the same as the other tables so I can join the data.
- I should also add another Index price for Media Companies along side the stocks to control for that variable.
</span>

<span style="color:red">

# Problems to be solved

1. The lions gate company stock that I am using only goes from 2017 and forward. Should we remove that company or no?
2. Also should we remove companies like ATT because that is not a media company rather they own a media company. Is that a problem?
</span>

# Setting File location

In [610]:
import os
import glob
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

os.chdir('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5813 - Economic Analytics I/Project Proposal/Data')

---------------

# 1. Loading and Modeling Stocks Data

In [611]:
# 1: Load stocks and add company names
file_paths = {
    'Amazon': 'Amazon Stock.csv',
    'Apple': 'Apple Stock.csv',
    'AT&T': 'AT&T Stock.csv',
    'Comcast': 'Comcast Stock.csv',
    'Lions Gate': 'Lions Gate Historical.csv',
    'Netflix': 'Netflix Stock Data.csv',
    'Paramount Global': 'Paramount Global Historical Prices.csv',
    'Sony': 'Sony Stock.csv',
    'Walt Disney': 'The Walt Disney Company Historical Prices.csv',
    'Warner Bros Discovery': 'Warner Bros Discovery.csv',
    'Vanguard 500': 'Vanguard 500 Historical.csv',
    'Media Company Index': 'Media Company index.csv'
}

# Load the datasets and convert 'Date' to datetime
stocks = {name: pd.read_csv(path, parse_dates=['Date']).assign(Company=name) for name, path in file_paths.items()}

# 2 & 3: Rename columns, and set 'Date' as index
for name, df in stocks.items():
    df.set_index('Date', inplace=True)
    df.rename(columns={col: f"{col}_{name}" for col in df.columns if col != 'Company'}, inplace=True)

# 4: Concatenate all dataframes along the columns (axis=1) using an outer join
all_stocks = pd.concat(stocks.values(), axis=1, join='outer')

# Before calculating the monthly averages, duplicate the 'Date' index into a new column
all_stocks['Date'] = all_stocks.index

# Now you can calculate monthly averages
numeric_columns = all_stocks.select_dtypes(include=['float64', 'int64']).columns
stocks_monthly_avg = all_stocks[numeric_columns].resample('M').mean()

# Add the 'Date' column back into stocks_monthly_avg from all_stocks before dropping the index
stocks_monthly_avg['Date'] = all_stocks['Date'].resample('M').first()

# Now we can safely rename the index
stocks_monthly_avg.index.rename('Date_index', inplace=True)

# Dropping unnecessary columns
keywords_to_drop = ['Low', 'High', 'Volume', 'Open', 'Adj Close']
columns_to_drop = stocks_monthly_avg.filter(regex='|'.join(keywords_to_drop)).columns
stocks_monthly_avg.drop(columns=columns_to_drop, inplace=True)

# Reset index if you want 'Date_index' to become a column instead of an index
stocks_monthly_avg.reset_index(inplace=True)

# Format both 'Date' and 'Date_index' columns to 'MM/YYYY'
stocks_monthly_avg['Date_index'] = stocks_monthly_avg['Date_index'].dt.strftime('%m/%Y')
stocks_monthly_avg['Date'] = stocks_monthly_avg['Date'].dt.strftime('%m/%Y')

/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_10839/2946492797.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stocks = {name: pd.read_csv(path, parse_dates=['Date']).assign(Company=name) for name, path in file_paths.items()}


In [612]:
# Display the head of the monthly averages DataFrame to verify
stocks_monthly_avg.tail()

,Date_index,Close_Amazon,Close_Apple,Close_AT&T,Close_Comcast,Close_Lions Gate,Close_Netflix,Close_Paramount Global,Close_Sony,Close_Walt Disney,Close_Warner Bros Discovery,Close_Vanguard 500,Close_Media Company Index,Date
738,07/2023,130.818001,192.411501,14.901000,42.766499,7.633000,440.328000,15.921500,92.706000,87.745499,12.820500,412.995499,220.294000,07/2023
739,08/2023,135.502175,181.083913,14.242609,45.855217,7.173043,424.058697,15.174783,84.963914,86.273913,13.201304,408.786955,224.720871,08/2023
740,09/2023,135.440001,177.002501,14.895000,45.141000,7.809500,407.321997,13.283000,84.426501,82.067001,11.313500,404.918498,226.654999,09/2023
741,10/2023,128.678999,176.895003,14.665000,43.841001,7.348000,384.119091,11.682273,83.581001,82.751999,10.437000,394.707001,226.143637,10/2023
742,11/2023,NaN,NaN,NaN,NaN,NaN,444.870710,12.545000,NaN,NaN,NaN,NaN,232.261250,11/2023


# Final Table name is <span style="color:red">stocks_monthly_avg</span>

---------------------

# 2. Loading and Modeling Economic Indicators

In [613]:
# Economic Indicators 
GDP = pd.read_csv('GDP.csv') # Billions of Dollars & Seasonally Adjusted Annual Rate
Unemployment_Rate = pd.read_csv('Unemployment Rate US.csv') # This is Percent & Seasonally Adjusted

In [614]:
#Renaming a column in the Unemployment table
Unemployment_Rate.rename(columns={'LRUNTTTTUSQ156S': 'Unemployment_rate'}, inplace=True)
# Unemployment_Rate.head()

# Convert the dictionaries to DataFrames
GDP = pd.DataFrame(GDP)
Unemployment = pd.DataFrame(Unemployment_Rate)

# Merge the DataFrames on the 'DATE' column
Economic_Indicators = GDP.merge(Unemployment, on='DATE', how='outer')

# Economic_Indicators.to_csv('file_name.csv')

In [615]:
Economic_Indicators.tail()

,DATE,GDP,Unemployment_rate
301,2022-04-01,25544.273,3.600000
302,2022-07-01,25994.639,3.566667
303,2022-10-01,26408.405,3.600000
304,2023-01-01,26813.601,3.500000
305,2023-04-01,27063.012,3.566667


In [616]:
# Convert 'DATE' to datetime format if it's not already
Economic_Indicators['DATE'] = pd.to_datetime(Economic_Indicators['DATE'])

# Format 'DATE' as 'MM/YYYY'
Economic_Indicators['DATE'] = Economic_Indicators['DATE'].dt.strftime('%m/%Y')

# Now 'DATE' will be in the 'MM/YYYY' format

In [617]:
# Renaming the 'DATE' column to 'Date'
Economic_Indicators.rename(columns={'DATE': 'Date'}, inplace=True)

In [618]:
# Convert 'Date' to datetime format for filtering (assuming it is in the format MM/YYYY)
Economic_Indicators['Date'] = pd.to_datetime(Economic_Indicators['Date'], format='%m/%Y')

# Filter the DataFrame for dates greater than or equal to January 2014
Economic_Indicators = Economic_Indicators[Economic_Indicators['Date'] >= '2013-12-30']

# Convert 'Date' back to 'MM/YYYY' format after filtering
Economic_Indicators['Date'] = Economic_Indicators['Date'].dt.strftime('%m/%Y')

In [619]:
Economic_Indicators.tail(20)

,Date,GDP,Unemployment_rate
286,07/2018,20798.730,3.766667
287,10/2018,20917.867,3.833333
288,01/2019,21104.133,3.866667
289,04/2019,21384.775,3.633333
290,07/2019,21694.282,3.633333
291,10/2019,21902.390,3.600000
292,01/2020,21706.513,3.800000
293,04/2020,19913.143,12.966667
294,07/2020,21647.640,8.833333
295,10/2020,22024.502,6.766667


In [620]:
# # Charting the Economic Indicators

# Economic_Indicators = pd.DataFrame(Economic_Indicators)

# # Convert 'DATE' to datetime if it's not already
# Economic_Indicators['Date'] = pd.to_datetime(Economic_Indicators['Date'])

# # Filter the DataFrame for years greater than or equal to 2000
# Economic_Indicators = Economic_Indicators[Economic_Indicators['Date'].dt.year >= 2000]

# # Plotting
# plt.figure(figsize=(14, 7))  # Set the figure size

# # Plot GDP
# plt.plot(Economic_Indicators['Date'], Economic_Indicators['GDP'], label='GDP', marker='o', color='blue')

# # Plot Unemployment Rate on a secondary y-axis
# ax2 = plt.gca().twinx()
# ax2.plot(Economic_Indicators['Date'], Economic_Indicators['Unemployment_rate'], label='Unemployment Rate', marker='o', color='red')

# # Adding labels and title
# plt.xlabel('Date')
# plt.ylabel('GDP', color='blue')
# ax2.set_ylabel('Unemployment Rate', color='red')
# plt.title('GDP and Unemployment Rate Over Time')

# # Rotate date labels for better readability
# plt.xticks(rotation=45)

# # Add legends
# plt.legend(loc='upper left')
# ax2.legend(loc='upper right')

# # Show a grid
# plt.grid(True)

# # Show the plot
# plt.tight_layout()  # Adjust the padding between and around subplots.
# plt.show()

# Final Table name is <span style="color:red">Economic_Indicators</span>

--------

# 3. Loading and Modeling all IMDB datasets

<a name="_page0_x69.69_y423.22"></a>**IMDb Dataset Legend**

<Details>

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

<a name="_page0_x60.71_y555.04"></a>**title.akas.tsv.gz**

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title <a name="_page1_x60.71_y183.54"></a>**title.basics.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.002.png) endYear (YYYY) – TV Series end year. ‘\N’ for all other title types ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.003.png) runtimeMinutes – primary runtime of the title, in minutes ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.004.png) genres (string array) – includes up to three genres associated with the title

<a name="_page1_x60.71_y549.05"></a>**title.crew.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- directors (array of nconsts) - director(s) of the given title
- writers (array of nconsts) – writer(s) of the given title

<a name="_page1_x60.71_y680.87"></a>**title.episode.tsv.gz**

- tconst (string) - alphanumeric identifier of episode
- parentTconst (string) - alphanumeric identifier of the parent TV Series
- seasonNumber (integer) – season number the episode belongs to
- episodeNumber (integer) – episode number of the tconst in the TV series

<a name="_page2_x60.71_y99.65"></a>**title.principals.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'
- characters (string) - the name of the character played if applicable, else '\N'

<a name="_page2_x60.71_y321.35"></a>**title.ratings.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

<a name="_page2_x60.71_y453.18"></a>**name.basics.tsv.gz**

- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for



In [621]:
# IMDB Data -- More about the dataset below in markdown

IMDb_Basics = pd.read_csv('IMDb Basics.tsv', sep='\t')
IMDb_Ratings = pd.read_csv('IMDb Ratings.tsv', sep='\t')

# Title_Principals = pd.read_csv('Title Principals.tsv', sep='\t')
# IMDb_Title_Akas = pd.read_csv('IMDb Title Akas.tsv', sep='\t')
# IMDb_Name_Basics = pd.read_csv('IMDb Name Basics.tsv', sep='\t')
IMDb_Title_Crew = pd.read_csv('IMDb Title Crew.tsv', sep='\t')
Title_Episode = pd.read_csv('Title Episode.tsv', sep='\t')

/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_10839/2929474124.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  IMDb_Basics = pd.read_csv('IMDb Basics.tsv', sep='\t')


In [622]:
Title_Episode.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


### 1. Cleaning IMDB_Basics
-----------

In [623]:
# Converting the data to different data types

IMDb_Basics['tconst'] = IMDb_Basics['tconst'].astype(str)
IMDb_Basics['primaryTitle'] = IMDb_Basics['primaryTitle'].astype(str)
IMDb_Basics['originalTitle'] = IMDb_Basics['originalTitle'].astype(str)
IMDb_Basics['isAdult'] = IMDb_Basics['isAdult'].astype(bool)

# Handle 'startYear'
IMDb_Basics['startYear'] = IMDb_Basics['startYear'].replace('\\N', np.nan)
IMDb_Basics['startYear'] = IMDb_Basics['startYear'].astype('float').astype('Int64')

# Handle 'endYear'
IMDb_Basics['endYear'] = IMDb_Basics['endYear'].replace('\\N', np.nan)
IMDb_Basics['endYear'] = IMDb_Basics['endYear'].astype('float').astype('Int64')

IMDb_Basics['runtimeMinutes'] = IMDb_Basics['runtimeMinutes'].astype(str)
IMDb_Basics['genres'] = IMDb_Basics['genres'].astype(str)

In [624]:
print(IMDb_Basics.dtypes)

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult             bool
startYear          Int64
endYear            Int64
runtimeMinutes    object
genres            object
dtype: object


In [625]:
# This code will display all the unique types of titles present in the titleType column of your DataFrame.
unique_title_types = IMDb_Basics['titleType'].unique()
print(unique_title_types)

['short' 'movie' 'tvShort' 'tvMovie' 'tvSeries' 'tvEpisode' 'tvMiniSeries'
 'tvSpecial' 'video' 'videoGame' 'tvPilot']


In [626]:
# Filtering DataFrame for rows where titleType is exactly 'Movie'
IMDb_Basics = IMDb_Basics[IMDb_Basics['titleType'] == 'movie']

In [627]:
IMDb_Basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,False,1894,<NA>,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,False,1897,<NA>,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,False,1905,<NA>,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,False,1906,<NA>,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,False,1907,<NA>,90,Drama


### 2. Cleaning IMDb_Ratings
------------

In [628]:
IMDb_Ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1892
3,tt0000004,5.5,178
4,tt0000005,6.2,2679


In [629]:
# Inspecting the data types
IMDb_Ratings.dtypes

tconst            object
averageRating    float64
numVotes           int64
dtype: object

## Merging all IMDb Datasets

In [630]:
# Merge datasets based on 'tconst'
merged_data = IMDb_Basics.merge(IMDb_Ratings, on='tconst', how='left')

In [631]:
# Remove rows where 'averageRating' is null
merged_data = merged_data.dropna(subset=['averageRating'])

# First, ensure that 'startYear' is of a numeric type, converting any non-numeric values to NaN
merged_data['startYear'] = pd.to_numeric(merged_data['startYear'], errors='coerce')

# Then, filter out rows where 'startYear' is not between 2014 and 2023
IMDB_dataset = merged_data[(merged_data['startYear'] >= 2014) & (merged_data['startYear'] <= 2023)]

In [632]:
IMDB_dataset.tail()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
659887,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,False,2020,<NA>,84,Thriller,5.8,1445.0
659888,tt9916362,movie,Coven,Akelarre,False,2020,<NA>,92,"Drama,History",6.4,5569.0
659889,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,False,2019,<NA>,\N,"Adventure,History,War",3.5,17.0
659890,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,False,2019,<NA>,123,Drama,8.6,7.0
659895,tt9916730,movie,6 Gunn,6 Gunn,False,2017,<NA>,116,Drama,7.6,11.0


# Final Table name is <span style="color:red">IMDB_dataset</span>

# 4. Loading and Modeling Movie Revenue dataset

In [633]:
# Define the path to the files
path_to_files = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5813 - Economic Analytics I/Project Proposal/Data/Gross Data/'

# Initialize an empty DataFrame for the Revenues_dataset
Revenues_dataset = pd.DataFrame()

# Define a helper function to extract the year from the filename
def get_year_from_filename(file_name):
    for month in ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]:
        if month in file_name:
            return file_name.split(month)[1].strip('.csv').strip()
    return None

# Loop through each CSV file in the directory
for file_name in os.listdir(path_to_files):
    if file_name.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(path_to_files, file_name)
        # Read the CSV file into a temporary DataFrame
        temp_df = pd.read_csv(file_path)
        # Extract the year from the file name
        year = get_year_from_filename(file_name)
        if year:
            # Assume that 'Release Date' column contains the day and abbreviated month (e.g., '17-Feb')
            # If day is missing, we'll prepend '01-' to use as a placeholder
            temp_df['Release Date'] = temp_df['Release Date'].str.extract(r'(\d+)-[A-Za-z]+', expand=False).fillna('01') + '-' + temp_df['Release Date'].str.extract(r'(\d+-)?([A-Za-z]+)', expand=False)[1] + '-' + year
            # Convert 'Release Date' to datetime format
            temp_df['Release Date'] = pd.to_datetime(temp_df['Release Date'], format='%d-%b-%Y', errors='coerce')
            # Drop rows where 'Release Date' could not be parsed
            temp_df.dropna(subset=['Release Date'], inplace=True)
            # Format 'Release Date' as 'MM/YYYY'
            temp_df['Release Date'] = temp_df['Release Date'].dt.strftime('%m/%Y')
            # Append to the main DataFrame
            Revenues_dataset = pd.concat([Revenues_dataset, temp_df], ignore_index=True)

# Remove rows with NaN 'Release Date' before dropping duplicates
Revenues_dataset.dropna(subset=['Release Date'], inplace=True)

# Drop duplicates based on the 'Release' column
Revenues_dataset = Revenues_dataset.drop_duplicates(subset='Release')

# Drop the unnecessary 'Rank' and 'Gross' columns
Revenues_dataset.drop(['Rank', 'Gross'], axis=1, inplace=True)

# Drop rows with null values in 'Total Gross'
Revenues_dataset = Revenues_dataset.dropna(subset=['Total Gross'])

In [634]:
Revenues_dataset.head(10)

,Release,Theaters,Total Gross,Release Date,Distributor
0,It Chapter Two,"4,570","$211,593,228",09/2019,Warner Bros.
1,Hustlers,"3,525","$104,963,598",09/2019,STX Entertainment
2,Downton Abbey,"3,548","$96,853,865",09/2019,Focus Features
3,Ad Astra,"3,460","$50,188,370",09/2019,Twentieth Century Fox
4,Rambo: Last Blood,"3,618","$44,819,352",09/2019,Lions Gate Films
5,Angel Has Fallen,"3,336","$69,030,436",08/2019,Lions Gate Films
6,Good Boys,"3,458","$83,140,306",08/2019,Universal Pictures
7,Abominable,"4,248","$60,716,390",09/2019,Universal Pictures
8,The Lion King,"4,802","$543,638,043",07/2019,Walt Disney Studios Motion Pictures
9,Overcomer,"2,293","$34,746,945",08/2019,Affirm Films


# Final Table name is <span style="color:red">Revenues_dataset</span>

-----

# MERGING ALL DATASETS
The showdown begins

### 1. Merging Stocks and economic indicators first

In [635]:
# Convert 'Date' columns in both DataFrames to datetime if they are not already
Economic_Indicators['Date'] = pd.to_datetime(Economic_Indicators['Date'], format='%m/%Y')
stocks_monthly_avg['Date'] = pd.to_datetime(stocks_monthly_avg['Date'], format='%m/%Y')

# Set 'Date' as index for the resampling operation
Economic_Indicators.set_index('Date', inplace=True)

# Ensure the Economic Indicators DataFrame covers the full date range of the stocks DataFrame
# This will provide a value for each month that can be forward filled
all_months = pd.date_range(start=stocks_monthly_avg['Date'].min(), 
                           end=stocks_monthly_avg['Date'].max(), 
                           freq='MS')
Economic_Indicators = Economic_Indicators.reindex(all_months)

# Resample Economic_Indicators to have rows for each month, forward filling the data
Economic_Indicators.ffill(inplace=True)

# Now, ensure the stocks_monthly_avg DataFrame has 'Date' as an index
stocks_monthly_avg.set_index('Date', inplace=True)

# Join the two DataFrames on the 'Date' index
combined_df = stocks_monthly_avg.join(Economic_Indicators, how='left')

# Reset the index to turn 'Date' back into a column
combined_df.reset_index(inplace=True)

# Convert 'Date' back to 'MM/YYYY' format
combined_df['Date'] = combined_df['Date'].dt.strftime('%m/%Y')

In [636]:
combined_df.tail(10)

,Date,Date_index,Close_Amazon,Close_Apple,Close_AT&T,Close_Comcast,Close_Lions Gate,Close_Netflix,Close_Paramount Global,Close_Sony,Close_Walt Disney,Close_Warner Bros Discovery,Close_Vanguard 500,Close_Media Company Index,GDP,Unemployment_rate
733,02/2023,02/2023,99.214211,150.968421,19.357895,38.774736,9.305789,349.097894,23.127895,87.719474,106.656842,15.227895,374.081051,182.205790,26813.601,3.500000
734,03/2023,03/2023,96.546956,154.964782,18.606522,36.351305,9.271304,312.603046,20.953478,86.732608,96.303913,14.612174,364.181304,181.042174,26813.601,3.500000
735,04/2023,04/2023,103.341579,165.045790,18.956842,38.105263,10.471053,333.743683,22.149474,90.848421,99.656316,14.106316,377.509997,193.282105,27063.012,3.566667
736,05/2023,05/2023,111.939547,172.622728,16.560000,40.292273,10.160455,347.460003,16.552727,94.433182,94.403181,12.341818,380.235458,202.772272,27063.012,3.566667
737,06/2023,06/2023,126.380476,184.283333,15.784762,40.574762,8.937619,421.458093,15.940000,95.760953,90.619047,12.622381,399.098572,214.558095,27063.012,3.566667
738,07/2023,07/2023,130.818001,192.411501,14.901000,42.766499,7.633000,440.328000,15.921500,92.706000,87.745499,12.820500,412.995499,220.294000,27063.012,3.566667
739,08/2023,08/2023,135.502175,181.083913,14.242609,45.855217,7.173043,424.058697,15.174783,84.963914,86.273913,13.201304,408.786955,224.720871,27063.012,3.566667
740,09/2023,09/2023,135.440001,177.002501,14.895000,45.141000,7.809500,407.321997,13.283000,84.426501,82.067001,11.313500,404.918498,226.654999,27063.012,3.566667
741,10/2023,10/2023,128.678999,176.895003,14.665000,43.841001,7.348000,384.119091,11.682273,83.581001,82.751999,10.437000,394.707001,226.143637,27063.012,3.566667
742,11/2023,11/2023,NaN,NaN,NaN,NaN,NaN,444.870710,12.545000,NaN,NaN,NaN,NaN,232.261250,27063.012,3.566667


### 2. Merging Revenue and IMDb datasets

In [637]:
# Merge the DataFrames on the movie titles
combined2_df = pd.merge(Revenues_dataset, IMDB_dataset, left_on='Release', right_on='primaryTitle')
combined2_df.head()

,Release,Theaters,Total Gross,Release Date,Distributor,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,It Chapter Two,"4,570","$211,593,228",09/2019,Warner Bros.,tt7349950,movie,It Chapter Two,It Chapter Two,False,2019,<NA>,169,"Drama,Fantasy,Horror",6.5,289707.0
1,Hustlers,"3,525","$104,963,598",09/2019,STX Entertainment,tt3512976,movie,Hustlers,Hustlers,False,2014,<NA>,105,Drama,7.8,30.0
2,Hustlers,"3,525","$104,963,598",09/2019,STX Entertainment,tt5503686,movie,Hustlers,Hustlers,False,2019,<NA>,110,"Comedy,Crime,Drama",6.3,105310.0
3,Downton Abbey,"3,548","$96,853,865",09/2019,Focus Features,tt6398184,movie,Downton Abbey,Downton Abbey,False,2019,<NA>,122,"Drama,Romance",7.4,60884.0
4,Ad Astra,"3,460","$50,188,370",09/2019,Twentieth Century Fox,tt2935510,movie,Ad Astra,Ad Astra,False,2019,<NA>,123,"Adventure,Drama,Mystery",6.5,253317.0


## Combining combined2_df and combined_df

In [638]:
# Convert the 'Date' and 'Release Date' columns to datetime if they are not already
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%m/%Y')
combined2_df['Release Date'] = pd.to_datetime(combined2_df['Release Date'], format='%m/%Y')

# Merge the DataFrames on the 'Date' and 'Release Date' columns
project_dataset = pd.merge(combined_df, combined2_df, left_on='Date', right_on='Release Date')

# Convert 'Date' back to 'MM/YYYY' format
project_dataset['Date'] = project_dataset['Date'].dt.strftime('%m/%Y')

## Cleaning the Project_dataset

In [639]:
project_dataset.tail()

,Date,Date_index,Close_Amazon,Close_Apple,Close_AT&T,Close_Comcast,Close_Lions Gate,Close_Netflix,Close_Paramount Global,Close_Sony,Close_Walt Disney,Close_Warner Bros Discovery,Close_Vanguard 500,Close_Media Company Index,GDP,Unemployment_rate,Release,Theaters,Total Gross,Release Date,Distributor,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
6223,06/2023,06/2023,126.380476,184.283333,15.784762,40.574762,8.937619,421.458093,15.94,95.760953,90.619047,12.622381,399.098572,214.558095,27063.012,3.566667,Revoir Paris,6,"$50,926",2023-06-01,Music Box Films,tt15329224,movie,Revoir Paris,Revoir Paris,False,2022,<NA>,105,Drama,6.8,2414.0
6224,06/2023,06/2023,126.380476,184.283333,15.784762,40.574762,8.937619,421.458093,15.94,95.760953,90.619047,12.622381,399.098572,214.558095,27063.012,3.566667,Falcon Lake,10,"$12,768",2023-06-01,Yellow Veil Pictures,tt11448830,movie,Falcon Lake,Falcon Lake,False,2022,<NA>,100,"Drama,Fantasy,Romance",7.2,1673.0
6225,06/2023,06/2023,126.380476,184.283333,15.784762,40.574762,8.937619,421.458093,15.94,95.760953,90.619047,12.622381,399.098572,214.558095,27063.012,3.566667,Loren & Rose,3,"$10,513",2023-06-01,Wise Lars,tt10358566,movie,Loren & Rose,Loren & Rose,False,2022,<NA>,83,Drama,7.8,35.0
6226,06/2023,06/2023,126.380476,184.283333,15.784762,40.574762,8.937619,421.458093,15.94,95.760953,90.619047,12.622381,399.098572,214.558095,27063.012,3.566667,Kings of L.A.,1,"$5,880",2023-06-01,Indican Pictures,tt11154884,movie,Kings of L.A.,Kings of L.A.,False,2023,<NA>,120,"Action,Comedy,Crime",8.2,33.0
6227,06/2023,06/2023,126.380476,184.283333,15.784762,40.574762,8.937619,421.458093,15.94,95.760953,90.619047,12.622381,399.098572,214.558095,27063.012,3.566667,Millie Lies Low,3,"$11,265",2023-06-01,Film Movement,tt14390304,movie,Millie Lies Low,Millie Lies Low,False,2021,<NA>,100,"Comedy,Drama",5.7,166.0


In [644]:
project_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6228 entries, 0 to 6227
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         6228 non-null   object        
 1   Date_index                   6228 non-null   object        
 2   Close_Amazon                 6228 non-null   float64       
 3   Close_Apple                  6228 non-null   float64       
 4   Close_AT&T                   6228 non-null   float64       
 5   Close_Comcast                6228 non-null   float64       
 6   Close_Lions Gate             4182 non-null   float64       
 7   Close_Netflix                6228 non-null   float64       
 8   Close_Paramount Global       6228 non-null   float64       
 9   Close_Sony                   6228 non-null   float64       
 10  Close_Walt Disney            6228 non-null   float64       
 11  Close_Warner Bros Discovery  6228 non-null 

In [657]:
project_dataset.columns

Index(['Date', 'Close_Amazon', 'Close_Apple', 'Close_AT&T', 'Close_Comcast',
       'Close_Lions Gate', 'Close_Netflix', 'Close_Paramount Global',
       'Close_Sony', 'Close_Walt Disney', 'Close_Warner Bros Discovery',
       'Close_Vanguard 500', 'Close_Media Company Index', 'GDP',
       'Unemployment_rate', 'Release', 'Theaters', 'Total Gross',
       'Release Date', 'Distributor', 'primaryTitle', 'originalTitle',
       'isAdult', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes'],
      dtype='object')

In [670]:
# Convert 'Total Gross' to numeric by removing '$' and ',' then converting to float
# Use a raw string for the regular expression
project_dataset['Total Gross'] = project_dataset['Total Gross'].replace(r'[\$,]', '', regex=True).astype(float)

# Filter out rows where 'Total Gross' is less than 4 million
project_dataset = project_dataset[project_dataset['Total Gross'] >= 4000000]

# Remove specified columns
columns_to_remove = ['startYear', 'titleType', 'tconst', 'Date_index', 'endYear']
project_dataset = project_dataset.drop(columns=columns_to_remove, errors='ignore')  # errors='ignore' to avoid issues if some columns don't exist# Check final DataFrame


In [671]:
project_dataset.head()

,Date,Close_Amazon,Close_Apple,Close_AT&T,Close_Comcast,Close_Lions Gate,Close_Netflix,Close_Paramount Global,Close_Sony,Close_Walt Disney,Close_Warner Bros Discovery,Close_Vanguard 500,Close_Media Company Index,GDP,Unemployment_rate,Release,Theaters,Total Gross,Release Date,Distributor,primaryTitle,originalTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes
2,01/2014,19.743167,19.194490,25.557834,26.469762,NaN,51.304082,60.758572,17.042381,74.264285,42.071684,166.779048,150.727619,17197.738,6.666667,The Nut Job,"3,472",64251541.0,2014-01-01,Open Road Films (II),The Nut Job,The Nut Job,False,85,"Adventure,Animation,Comedy",5.7,29715.0
3,01/2014,19.743167,19.194490,25.557834,26.469762,NaN,51.304082,60.758572,17.042381,74.264285,42.071684,166.779048,150.727619,17197.738,6.666667,Ride Along,"2,867",134938200.0,2014-01-01,Universal Pictures,Ride Along,Ride Along,False,99,"Action,Comedy,Crime",6.1,120592.0
5,01/2014,19.743167,19.194490,25.557834,26.469762,NaN,51.304082,60.758572,17.042381,74.264285,42.071684,166.779048,150.727619,17197.738,6.666667,Jack Ryan: Shadow Recruit,"3,387",50577412.0,2014-01-01,Paramount Pictures,Jack Ryan: Shadow Recruit,Jack Ryan: Shadow Recruit,False,105,"Action,Drama,Thriller",6.2,136516.0
18,02/2014,17.716842,18.806428,24.574654,26.391579,NaN,61.631278,63.270527,16.936316,77.578948,41.576796,166.606315,146.725789,17197.738,6.666667,The Lego Movie,"3,890",257760692.0,2014-02-01,Warner Bros.,The Lego Movie,The Lego Movie,False,100,"Action,Adventure,Animation",7.7,379091.0
19,02/2014,17.716842,18.806428,24.574654,26.391579,NaN,61.631278,63.270527,16.936316,77.578948,41.576796,166.606315,146.725789,17197.738,6.666667,Son of God,"3,271",59700064.0,2014-02-01,Twentieth Century Fox,Son of God,Son of God,False,138,"Biography,Drama,History",5.7,19455.0


In [673]:
import pandas as pd

# Assuming project_dataset is your DataFrame
# Get all unique values in the 'Distributor' column
unique_distributors = project_dataset['Distributor'].unique()

# Print the unique values
print(unique_distributors)


['Open Road Films (II)' 'Universal Pictures' 'Paramount Pictures'
 'Warner Bros.' 'Twentieth Century Fox'
 'Sony Pictures Entertainment (SPE)' 'Screen Gems' 'Lions Gate Films' '-'
 'Freestyle Releasing' 'Walt Disney Studios Motion Pictures'
 'TriStar Pictures' 'The Weinstein Company' 'Focus Features'
 'STX Entertainment' 'Roadside Attractions'
 'Entertainment Studios Motion Pictures' 'A24' 'Pantelion Films' 'Neon'
 'United Artists Releasing' 'Metro-Goldwyn-Mayer (MGM)'
 'Annapurna Pictures' '20th Century Studios' 'FUNimation Entertainment'
 'Columbia Pictures']


All the companies I have
    'Amazon': 'Amazon Stock.csv',
    'Apple': 'Apple Stock.csv',
    'AT&T': 'AT&T Stock.csv',
    'Comcast': 'Comcast Stock.csv',
    'Lions Gate': 'Lions Gate Historical.csv',
    'Netflix': 'Netflix Stock Data.csv',
    'Paramount Global': 'Paramount Global Historical Prices.csv',
    'Sony': 'Sony Stock.csv',
    'Walt Disney': 'The Walt Disney Company Historical Prices.csv',
    'Warner Bros Discovery': 'Warner Bros Discovery.csv',

'Open Road Films (II)'- Remove
'-'- Remove
'Freestyle Releasing'- Remove
'The Weinstein Company'- Remove
'STX Entertainment' - Remove
'Entertainment Studios Motion Pictures': Remove
'A24'- Remove
'Pantelion Films'- Remove
'Neon'- Remove
'United Artists Releasing' - Remove
'Metro-Goldwyn-Mayer (MGM)'
'Annapurna Pictures'

'Universal Pictures':Owned by Comcast
'Paramount Pictures':Owned Paramount Global
'Warner Bros.':Warner Bros Discovery
'Twentieth Century Fox':Walt Disney
'Sony Pictures Entertainment (SPE)':Sony
'Screen Gems':Sony
'Lions Gate Films':Lions Gate
'Walt Disney Studios Motion Pictures': Walt Disney
'TriStar Pictures': Sony
'Focus Features': Comcast
'Roadside Attractions': Lionsgate
'20th Century Studios': Disney
'FUNimation Entertainment': Sony
'Columbia Pictures': Sony

In [672]:
project_dataset.count()

Date                           660
Close_Amazon                   660
Close_Apple                    660
Close_AT&T                     660
Close_Comcast                  660
Close_Lions Gate               403
Close_Netflix                  660
Close_Paramount Global         660
Close_Sony                     660
Close_Walt Disney              660
Close_Warner Bros Discovery    660
Close_Vanguard 500             660
Close_Media Company Index      660
GDP                            660
Unemployment_rate              660
Release                        660
Theaters                       660
Total Gross                    660
Release Date                   660
Distributor                    660
primaryTitle                   660
originalTitle                  660
isAdult                        660
runtimeMinutes                 660
genres                         660
averageRating                  660
numVotes                       660
dtype: int64

In [684]:
# Filtering dataframes for a specific values
filtered_df = project_dataset[project_dataset['Distributor'].str.contains('A24', case=False, na=False)]
filtered_df.head()

,Date,Close_Amazon,Close_Apple,Close_AT&T,Close_Comcast,Close_Lions Gate,Close_Netflix,Close_Paramount Global,Close_Sony,Close_Walt Disney,Close_Warner Bros Discovery,Close_Vanguard 500,Close_Media Company Index,GDP,Unemployment_rate,Release,Theaters,Total Gross,Release Date,Distributor,primaryTitle,originalTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes
2847,11/2017,56.990405,43.101548,25.893756,36.382857,29.898572,195.223333,56.331905,46.381905,102.520952,17.448095,238.155712,146.351429,20037.088,4.166667,Lady Bird,"1,557",48958273.0,2017-11-01,A24,Lady Bird,Lady Bird,False,94,"Comedy,Drama",7.4,314716.0
3331,06/2018,84.941190,47.155357,24.606172,32.542381,23.842381,384.133336,54.151905,49.668571,104.537142,25.531429,253.329047,146.849524,20580.912,3.933333,Hereditary,"2,998",44069456.0,2018-06-01,A24,Hereditary,Hereditary,False,127,"Drama,Horror,Mystery",7.3,356407.0
4617,12/2019,89.288643,69.131428,29.085743,43.860952,9.188095,314.603809,40.386667,66.864285,146.758095,32.517143,291.746666,180.069048,21902.390,3.600000,Uncut Gems,"2,686",50023780.0,2019-12-01,A24,Uncut Gems,Uncut Gems,False,135,"Crime,Drama,Thriller",7.4,305355.0
5528,03/2022,154.280480,165.310435,17.709838,46.536956,14.590870,366.791741,35.850869,101.890000,138.193479,26.141304,403.018695,229.318696,25029.116,3.800000,Everything Everywhere All at Once,"2,220",77191785.0,2022-03-01,A24,Everything Everywhere All at Once,Everything Everywhere All at Once,False,139,"Action,Adventure,Comedy",7.8,483015.0


# Project Dataset name is <span style="color:Blue">Revenues_dataset</span>